In [6]:
import requests
import bs4 as bs
import pandas as pd
import numpy as np
import re
import json

In [19]:
start = 9000 # put your number here
end = start + 2000
batch = 100

In [21]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0',
    'From': 'data-x@gmail.com' 
}

price_labels = ['index', 'name', 'date', 'price']

def populate_row(row_index, data, soup):
    parms = soup.find(class_="basic-parms-mod")
    results = [i.contents[0] for i in parms.find_all("dd")]
    data.loc[row_index, 'property_type'] = results[0]
    data.loc[row_index, 'property_costs'] = results[1]
    data.loc[row_index, 'total_area'] = results[2]
    data.loc[row_index, 'number_of_units'] = results[3]
    data.loc[row_index, 'year_built'] = results[4]
    data.loc[row_index, 'parking_spots'] = results[5]
    data.loc[row_index, 'volume_rate'] = results[6]
    data.loc[row_index, 'green_rating'] = results[7]
    data.loc[row_index, 'developer_company'] = results[8]
    data.loc[row_index, 'property_management_company'] =results[9]

def add_price(row_index, data, data_prices, soup):
    lookbehind = '(?<=new ajk.chart.priceTrend\(\{)'
    getall = '[\s\S]+?'
    lookahead = '(?=}\);)'
    m = re.findall(lookbehind + getall + lookahead, str(soup))
    lookbehind = '(?<=data : {)'
    getall = '[\s\S]+?'
    lookahead = '(?=},\n)'
    m2 = re.findall(lookbehind + getall + lookahead, str(m[1]))
    price_history = json.loads('{' + m2[0] + '}')['community']
    list(price_history[0].items())
    data_list = [[row_index, data['address'][row_index], list(p.items())[0][0], list(p.items())[0][1]] for p in price_history]
    df = pd.DataFrame.from_records(data_list, columns=price_labels)
    return data_prices.append(df, ignore_index=True)
    
def populate_batch_row(start = 0, batch_size = 1000):
    data = pd.read_csv('data_populated_all_fixed.csv', index_col=0)
    try:
        data_prices = pd.read_csv('data_prices.csv', index_col=0)
    except pd.errors.EmptyDataError:
        data_prices = pd.DataFrame()
    end = start + batch_size

    for i in range(start, end):
        try:
            link = data['link'][i]
            source = requests.get(link, headers=headers).content
            soup = bs.BeautifulSoup(source, 'html.parser')
            populate_row(i, data, soup)
            data_prices = add_price(i, data, data_prices, soup)
        except requests.ConnectionError:
            continue
        print("done with row ", i)
    data.to_csv('data_populated_all_fixed.csv')
    data_prices.to_csv('data_prices.csv')
    return (data, data_prices)

data, data_prices = None, None

for i in range(start, end, batch):
    data, data_prices = populate_batch_row(i, batch)

done with row  9000
done with row  9001
done with row  9002
done with row  9003
done with row  9004
done with row  9005
done with row  9006
done with row  9007
done with row  9008
done with row  9009
done with row  9010
done with row  9011
done with row  9012
done with row  9013
done with row  9014
done with row  9015


AttributeError: 'NoneType' object has no attribute 'find_all'